## Environment Setup

In [1]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
  

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your key goes here>'
os.environ['MISTRAL_API_KEY'] = '<your key goes here>'

In [3]:
questions = [ "Who is the main character in Moby Dick?",
              "Who is Ishmael, and why is he important?",
              "What role does Queequeg play in Moby Dick?"]

## Indexing

### Load the text

In [4]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)

1412

### Indexing to ChromaDB

In [6]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=MistralAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(



## Retrieval

### Retrieval of relevant documents

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
docs = retriever.get_relevant_documents(questions[0])
len(docs)
docs[0]

<ipython-input-7-7c4c69bbb4ef>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(questions[0])


Document(metadata={'source': 'https://www.gutenberg.org/cache/epub/2701/pg2701.txt'}, page_content='him—him—” faltering hard for a word, and screwing his hand round and\r\nround as though uncorking a bottle—“like him—him—”\r\n\r\n“Corkscrew!” cried Ahab, “aye, Queequeg, the harpoons lie all twisted\r\nand wrenched in him; aye, Daggoo, his spout is a big one, like a whole\r\nshock of wheat, and white as a pile of our Nantucket wool after the\r\ngreat annual sheep-shearing; aye, Tashtego, and he fan-tails like a\r\nsplit jib in a squall. Death and devils! men, it is Moby Dick ye have\r\nseen—Moby Dick—Moby Dick!”\r\n\r\n“Captain Ahab,” said Starbuck, who, with Stubb and Flask, had thus far\r\nbeen eyeing his superior with increasing surprise, but at last seemed\r\nstruck with a thought which somewhat explained all the wonder. “Captain\r\nAhab, I have heard of Moby Dick—but it was not Moby Dick that took off\r\nthy leg?”\r\n\r\n“Who told thee that?” cried Ahab; then pausing, “Aye, Starbuc

# Generation

## Initializing the Prompt template for Generator

In [8]:
from langchain_mistralai import ChatMistralAI
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

## Init LLM

In [9]:
# LLM
llm = ChatMistralAI(model_name="mistral-large-latest", temperature=0)

## Init simple Chain

In [10]:
# Chain
chain = prompt | llm

## Invoking the trivial chain for a given user question

In [11]:
# Run
chain.invoke({"context":docs,"question":questions[0]})

AIMessage(content='The main character in "Moby Dick" is Captain Ahab.', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 397, 'total_tokens': 411, 'completion_tokens': 14}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-8eb76641-487b-4c9f-86b5-c8b67ecd112d-0', usage_metadata={'input_tokens': 397, 'output_tokens': 14, 'total_tokens': 411})

## Preparing and calling RAG chain (with parameter, parser)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
for question in questions:
    print("-------------")
    print(question)
    print(rag_chain.invoke(question))

-------------
Who is the main character in Moby Dick?
The main character in "Moby Dick" is Captain Ahab.
-------------
Who is Ishmael, and why is he important?
In the given context, Ishmael is the young man who is speaking to Captain Peleg. He is important in this scene because he is in the process of signing up for a whaling voyage, and he is also inquiring about bringing a friend to sign up as well.
-------------
What role does Queequeg play in Moby Dick?
Queequeg is a harpooneer.
